In [1]:
import tensorflow as tf
from tensorflow.keras import layers, Model
import numpy as np
import cv2
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import pickle

class InventoryVerificationSystem:
    def __init__(self):
        self.feature_extractor = self._build_feature_extractor()
        self.similarity_threshold = 0.85
        self.fraud_detection_model = self._build_fraud_detector()

    def _build_feature_extractor(self):
        """CNN untuk extract features dari gambar sampah"""
        base_model = tf.keras.applications.EfficientNetB0(
            weights='imagenet',
            include_top=False,
            input_shape=(224, 224, 3)
        )
        base_model.trainable = False

        model = tf.keras.Sequential([
            base_model,
            layers.GlobalAveragePooling2D(),
            layers.Dense(512, activation='relu'),
            layers.Dropout(0.3),
            layers.Dense(256, activation='relu'),
            layers.L2Normalize()  # Untuk similarity comparison
        ])

        return model

    def _build_fraud_detector(self):
        """Model untuk detect fraud patterns"""
        inputs = tf.keras.Input(shape=(10,))  # Features: similarity, time, location, etc.

        x = layers.Dense(128, activation='relu')(inputs)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.4)(x)

        x = layers.Dense(64, activation='relu')(x)
        x = layers.BatchNormalization()(x)
        x = layers.Dropout(0.3)(x)

        x = layers.Dense(32, activation='relu')(x)
        outputs = layers.Dense(1, activation='sigmoid')(x)  # Fraud probability

        model = Model(inputs, outputs)
        model.compile(
            optimizer='adam',
            loss='binary_crossentropy',
            metrics=['accuracy', 'precision', 'recall']
        )

        return model

    def extract_features(self, image):
        """Extract features dari gambar"""
        if isinstance(image, str):
            image = cv2.imread(image)

        # Preprocessing
        image = cv2.resize(image, (224, 224))
        image = image.astype('float32') / 255.0
        image = np.expand_dims(image, axis=0)

        return self.feature_extractor(image)

    def calculate_similarity(self, features1, features2):
        """Hitung cosine similarity"""
        return tf.keras.utils.cosine_similarity(features1, features2).numpy()[0]

    def detect_fraud(self, inventory_image, collected_image, metadata):
        """Main fraud detection function"""
        # Extract features
        inv_features = self.extract_features(inventory_image)
        col_features = self.extract_features(collected_image)

        # Calculate similarity
        similarity = self.calculate_similarity(inv_features, col_features)

        # Prepare features for fraud detection
        fraud_features = np.array([[
            similarity,
            metadata.get('time_diff', 0),  # Waktu antara scan dan pickup
            metadata.get('location_diff', 0),  # Jarak lokasi
            metadata.get('user_history_score', 1.0),  # User reputation
            metadata.get('waste_type_match', 1.0),  # Type consistency
            metadata.get('quantity_ratio', 1.0),  # Quantity consistency
            metadata.get('lighting_score', 1.0),  # Image quality
            metadata.get('angle_consistency', 1.0),  # Angle similarity
            metadata.get('background_match', 1.0),  # Background consistency
            metadata.get('timestamp_validity', 1.0)  # Timestamp validation
        ]])

        # Predict fraud probability
        fraud_prob = self.fraud_detection_model.predict(fraud_features)[0][0]

        return {
            'is_valid': similarity > self.similarity_threshold and fraud_prob < 0.3,
            'similarity_score': float(similarity),
            'fraud_probability': float(fraud_prob),
            'confidence': 1 - abs(0.5 - fraud_prob) * 2,
            'details': {
                'visual_match': similarity > self.similarity_threshold,
                'fraud_risk': 'low' if fraud_prob < 0.3 else 'medium' if fraud_prob < 0.7 else 'high'
            }
        }

# Training function
def train_fraud_detector(model, training_data):
    """Train fraud detection model"""
    X_train, y_train = training_data

    # Class weights untuk handle imbalanced data
    class_weights = {0: 1, 1: 10}  # Fraud cases lebih sedikit

    history = model.fit(
        X_train, y_train,
        epochs=100,
        batch_size=32,
        validation_split=0.2,
        class_weight=class_weights,
        callbacks=[
            tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
            tf.keras.callbacks.ReduceLROnPlateau(patience=5)
        ]
    )

    return history

In [2]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd

class WasteDataPreprocessor:
    def __init__(self):
        self.augmentation = A.Compose([
            A.RandomRotate90(p=0.3),
            A.HorizontalFlip(p=0.3),
            A.RandomBrightnessContrast(p=0.3),
            A.GaussNoise(p=0.2),
            A.Blur(blur_limit=3, p=0.2),
            A.ColorJitter(p=0.3),
            A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ])

    def create_fraud_dataset(self, legitimate_pairs, fraud_scenarios):
        """Generate training dataset untuk fraud detection"""
        data = []

        # Legitimate cases
        for inv_img, col_img, metadata in legitimate_pairs:
            features = self.extract_all_features(inv_img, col_img, metadata)
            data.append(features + [0])  # Label: Not fraud

        # Fraud cases (synthetic + real if available)
        fraud_cases = self.generate_fraud_cases(legitimate_pairs)
        for features in fraud_cases:
            data.append(features + [1])  # Label: Fraud

        df = pd.DataFrame(data, columns=[
            'similarity', 'time_diff', 'location_diff', 'user_score',
            'waste_type_match', 'quantity_ratio', 'lighting_score',
            'angle_consistency', 'background_match', 'timestamp_validity', 'is_fraud'
        ])

        return df

    def generate_fraud_cases(self, legitimate_pairs):
        """Generate synthetic fraud scenarios"""
        fraud_cases = []

        for inv_img, col_img, metadata in legitimate_pairs:
            # Scenario 1: Different waste item
            fraud_metadata = metadata.copy()
            fraud_metadata['similarity'] = np.random.uniform(0.1, 0.4)
            fraud_metadata['waste_type_match'] = 0.2
            fraud_cases.append(self.extract_all_features(inv_img, col_img, fraud_metadata))

            # Scenario 2: Suspicious timing
            fraud_metadata = metadata.copy()
            fraud_metadata['time_diff'] = np.random.uniform(24, 168)  # 1-7 days
            fraud_metadata['user_score'] = 0.3
            fraud_cases.append(self.extract_all_features(inv_img, col_img, fraud_metadata))

            # Scenario 3: Location mismatch
            fraud_metadata = metadata.copy()
            fraud_metadata['location_diff'] = np.random.uniform(5, 50)  # 5-50km
            fraud_cases.append(self.extract_all_features(inv_img, col_img, fraud_metadata))

        return fraud_cases

In [3]:
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score
import matplotlib.pyplot as plt
import seaborn as sns

class ModelEvaluator:
    def __init__(self, model):
        self.model = model

    def comprehensive_evaluation(self, test_data, test_labels):
        """Comprehensive model evaluation"""
        predictions = self.model.predict(test_data)
        pred_binary = (predictions > 0.5).astype(int)

        # Metrics
        accuracy = accuracy_score(test_labels, pred_binary)
        precision, recall, f1, _ = precision_recall_fscore_support(test_labels, pred_binary, average='binary')
        auc_score = roc_auc_score(test_labels, predictions)

        # Confusion Matrix
        cm = confusion_matrix(test_labels, pred_binary)

        # Calculate business metrics
        false_positives = cm[0][1]  # Legitimate flagged as fraud
        false_negatives = cm[1][0]  # Fraud missed

        results = {
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'auc_score': auc_score,
            'false_positive_rate': false_positives / (cm[0][0] + cm[0][1]),
            'false_negative_rate': false_negatives / (cm[1][0] + cm[1][1]),
            'business_impact': {
                'users_incorrectly_flagged': false_positives,
                'fraud_cases_missed': false_negatives,
                'estimated_loss_prevented': false_negatives * 100  # Assume $100 per fraud
            }
        }

        return results

    def plot_performance(self, history, test_results):
        """Visualize model performance"""
        fig, axes = plt.subplots(2, 2, figsize=(15, 10))

        # Training history
        axes[0,0].plot(history.history['accuracy'], label='Train')
        axes[0,0].plot(history.history['val_accuracy'], label='Validation')
        axes[0,0].set_title('Model Accuracy')
        axes[0,0].legend()

        # Loss
        axes[0,1].plot(history.history['loss'], label='Train')
        axes[0,1].plot(history.history['val_loss'], label='Validation')
        axes[0,1].set_title('Model Loss')
        axes[0,1].legend()

        # Confusion Matrix
        sns.heatmap(test_results['confusion_matrix'], annot=True, ax=axes[1,0])
        axes[1,0].set_title('Confusion Matrix')

        # Metrics bar chart
        metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'AUC']
        values = [test_results[m.lower().replace('-', '_')] for m in metrics]
        axes[1,1].bar(metrics, values)
        axes[1,1].set_title('Performance Metrics')
        axes[1,1].set_ylim(0, 1)

        plt.tight_layout()
        plt.show()

In [5]:
# ===== TRAINING SCRIPT YANG DIPERBAIKI =====
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import matplotlib.pyplot as plt

class ImprovedDataGenerator:
    def __init__(self):
        self.noise_factor = 0.1

    def create_realistic_dataset(self, num_samples=10000):
        """Generate more realistic and noisy dataset"""
        print("🔄 Generating realistic training data with noise...")

        np.random.seed(42)  # Untuk reproducibility
        data = []

        # Legitimate cases (60% of data) - dengan variasi realistis
        for i in range(int(num_samples * 0.6)):
            # Base legitimate pattern dengan noise
            base_similarity = np.random.beta(9, 2)  # Mostly high, some variation
            noise = np.random.normal(0, 0.05)
            similarity = np.clip(base_similarity + noise, 0.3, 1.0)

            features = [
                similarity,
                np.random.gamma(2, 1.5),           # time_diff: realistic distribution
                np.random.gamma(1, 0.8),           # location_diff
                np.random.beta(7, 3),              # user_score with variation
                np.random.choice([0.7, 0.8, 0.9, 1.0], p=[0.1, 0.2, 0.3, 0.4]),
                np.random.normal(1.0, 0.15),       # quantity with noise
                np.random.beta(6, 4),              # lighting variations
                np.random.beta(7, 3),              # angle consistency
                np.random.beta(5, 5),              # background match varies
                np.random.choice([0.8, 0.9, 1.0], p=[0.2, 0.3, 0.5]),
                0  # legitimate
            ]

            # Add realistic constraints and noise
            features[1] = max(0.5, min(48, features[1]))  # 0.5-48 hours
            features[2] = max(0.1, min(10, features[2]))  # 0.1-10 km
            features[5] = max(0.7, min(1.3, features[5]))  # quantity ratio

            data.append(features)

        # Edge cases - legitimate but suspicious (15%)
        for i in range(int(num_samples * 0.15)):
            features = [
                np.random.uniform(0.65, 0.85),     # Lower similarity but still ok
                np.random.uniform(6, 24),          # Longer time but acceptable
                np.random.uniform(1, 5),           # Further distance
                np.random.uniform(0.4, 0.7),       # Lower user score
                np.random.choice([0.6, 0.7, 0.8], p=[0.3, 0.4, 0.3]),
                np.random.uniform(0.8, 1.2),
                np.random.uniform(0.4, 0.8),
                np.random.uniform(0.5, 0.8),
                np.random.uniform(0.3, 0.7),
                np.random.uniform(0.6, 1.0),
                0  # still legitimate
            ]
            data.append(features)

        # Clear fraud cases (20%)
        for i in range(int(num_samples * 0.2)):
            fraud_type = np.random.choice(['visual', 'temporal', 'behavioral', 'mixed'])

            if fraud_type == 'visual':
                features = [
                    np.random.uniform(0.1, 0.45),   # Very low similarity
                    np.random.gamma(2, 1),
                    np.random.gamma(1, 1),
                    np.random.beta(4, 6),
                    np.random.uniform(0.2, 0.5),    # Poor type match
                    np.random.uniform(0.5, 1.5),
                    np.random.uniform(0.3, 0.7),
                    np.random.uniform(0.2, 0.6),
                    np.random.uniform(0.2, 0.5),
                    np.random.uniform(0.5, 0.9),
                    1
                ]
            elif fraud_type == 'temporal':
                features = [
                    np.random.uniform(0.4, 0.7),
                    np.random.uniform(48, 336),     # 2-14 days
                    np.random.uniform(2, 20),
                    np.random.beta(2, 8),           # Very poor user score
                    np.random.uniform(0.4, 0.8),
                    np.random.uniform(0.7, 1.3),
                    np.random.uniform(0.3, 0.7),
                    np.random.uniform(0.4, 0.8),
                    np.random.uniform(0.3, 0.7),
                    np.random.uniform(0.1, 0.6),    # Poor timestamp validity
                    1
                ]
            elif fraud_type == 'behavioral':
                features = [
                    np.random.uniform(0.5, 0.8),
                    np.random.uniform(1, 12),
                    np.random.uniform(10, 100),     # Very far location
                    np.random.beta(1, 9),           # Terrible user score
                    np.random.uniform(0.3, 0.7),
                    np.random.uniform(0.3, 2.0),    # Suspicious quantity
                    np.random.uniform(0.2, 0.6),
                    np.random.uniform(0.3, 0.7),
                    np.random.uniform(0.1, 0.4),    # Very poor background
                    np.random.uniform(0.2, 0.7),
                    1
                ]
            else:  # mixed fraud
                features = [
                    np.random.uniform(0.2, 0.6),
                    np.random.uniform(12, 168),
                    np.random.uniform(3, 30),
                    np.random.beta(2, 8),
                    np.random.uniform(0.2, 0.6),
                    np.random.uniform(0.4, 1.8),
                    np.random.uniform(0.2, 0.6),
                    np.random.uniform(0.2, 0.6),
                    np.random.uniform(0.1, 0.5),
                    np.random.uniform(0.1, 0.6),
                    1
                ]

            data.append(features)

        # Ambiguous cases - hard to classify (5%)
        for i in range(int(num_samples * 0.05)):
            # These should be challenging for the model
            label = np.random.choice([0, 1])  # Random label for ambiguous
            features = [
                np.random.uniform(0.5, 0.75),    # Medium similarity
                np.random.uniform(8, 48),        # Medium time
                np.random.uniform(2, 8),         # Medium distance
                np.random.uniform(0.4, 0.7),     # Medium user score
                np.random.uniform(0.5, 0.8),
                np.random.uniform(0.8, 1.2),
                np.random.uniform(0.4, 0.8),
                np.random.uniform(0.4, 0.8),
                np.random.uniform(0.4, 0.8),
                np.random.uniform(0.6, 0.9),
                label
            ]
            data.append(features)

        # Shuffle data
        np.random.shuffle(data)

        columns = ['similarity', 'time_diff', 'location_diff', 'user_score',
                  'waste_type_match', 'quantity_ratio', 'lighting_score',
                  'angle_consistency', 'background_match', 'timestamp_validity', 'is_fraud']

        df = pd.DataFrame(data, columns=columns)

        print(f"✅ Realistic dataset created: {len(df)} samples")
        print(f"📊 Legitimate: {len(df[df['is_fraud']==0])}, Fraud: {len(df[df['is_fraud']==1])}")
        print(f"📈 Class distribution: {df['is_fraud'].value_counts(normalize=True)}")

        return df

def create_regularized_model():
    """Create model with proper regularization"""
    inputs = tf.keras.Input(shape=(10,), name='features')

    # Add input dropout
    x = tf.keras.layers.Dropout(0.1)(inputs)

    # Smaller layers with more regularization
    x = tf.keras.layers.Dense(64, activation='relu',
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    x = tf.keras.layers.Dense(32, activation='relu',
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(0.4)(x)

    x = tf.keras.layers.Dense(16, activation='relu',
                             kernel_regularizer=tf.keras.regularizers.l1_l2(l1=0.01, l2=0.01))(x)
    x = tf.keras.layers.Dropout(0.3)(x)

    # Output layer
    outputs = tf.keras.layers.Dense(1, activation='sigmoid', name='fraud_prob')(x)

    model = tf.keras.Model(inputs, outputs, name='RecycloFraudDetector_v2')

    # Use lower learning rate
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
        loss='binary_crossentropy',
        metrics=['accuracy', 'precision', 'recall']
    )

    return model

def improved_training():
    """Improved training with realistic expectations"""
    print("🚀 Starting IMPROVED Fraud Detection Training")
    print("="*60)

    # 1. Generate realistic data
    data_generator = ImprovedDataGenerator()
    df = data_generator.create_realistic_dataset(num_samples=10000)

    # 2. Feature analysis
    print("\n📊 Feature Analysis:")
    print(df.describe())

    # 3. Prepare data
    feature_columns = ['similarity', 'time_diff', 'location_diff', 'user_score',
                      'waste_type_match', 'quantity_ratio', 'lighting_score',
                      'angle_consistency', 'background_match', 'timestamp_validity']

    X = df[feature_columns].values
    y = df['is_fraud'].values

    # 4. Robust train/val/test split
    X_train, X_temp, y_train, y_temp = train_test_split(
        X, y, test_size=0.4, random_state=42, stratify=y
    )
    X_val, X_test, y_val, y_test = train_test_split(
        X_temp, y_temp, test_size=0.5, random_state=42, stratify=y_temp
    )

    # 5. Scaling
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_val_scaled = scaler.transform(X_val)
    X_test_scaled = scaler.transform(X_test)

    print(f"📊 Final data splits:")
    print(f"Train: {len(X_train)} ({np.mean(y_train):.3f} fraud rate)")
    print(f"Val: {len(X_val)} ({np.mean(y_val):.3f} fraud rate)")
    print(f"Test: {len(X_test)} ({np.mean(y_test):.3f} fraud rate)")

    # 6. Create model with regularization
    model = create_regularized_model()
    model.summary()

    # 7. Training with realistic expectations
    class_weight = {0: 1.0, 1: 2.0}  # Moderate class weight

    callbacks = [
        tf.keras.callbacks.EarlyStopping(
            monitor='val_loss',
            patience=15,
            restore_best_weights=True
        ),
        tf.keras.callbacks.ReduceLROnPlateau(
            monitor='val_loss',
            factor=0.5,
            patience=8,
            min_lr=1e-6
        )
    ]

    history = model.fit(
        X_train_scaled, y_train,
        epochs=200,
        batch_size=64,  # Larger batch size
        validation_data=(X_val_scaled, y_val),
        class_weight=class_weight,
        callbacks=callbacks,
        verbose=1
    )

    # 8. Final evaluation
    test_pred_prob = model.predict(X_test_scaled)
    test_pred = (test_pred_prob > 0.5).astype(int)

    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score

    results = {
        'accuracy': accuracy_score(y_test, test_pred),
        'precision': precision_score(y_test, test_pred),
        'recall': recall_score(y_test, test_pred),
        'f1_score': f1_score(y_test, test_pred),
        'auc_score': roc_auc_score(y_test, test_pred_prob),
    }

    print("\n" + "="*50)
    print("🎯 REALISTIC MODEL EVALUATION")
    print("="*50)
    print(f"✅ Accuracy:  {results['accuracy']:.4f} (Target: 0.85-0.95)")
    print(f"🎯 Precision: {results['precision']:.4f} (Target: 0.80-0.90)")
    print(f"📊 Recall:    {results['recall']:.4f} (Target: 0.75-0.90)")
    print(f"📈 F1-Score:  {results['f1_score']:.4f} (Target: 0.80-0.90)")
    print(f"📈 AUC Score: {results['auc_score']:.4f} (Target: 0.85-0.95)")

    # Confusion Matrix
    cm = confusion_matrix(y_test, test_pred)
    print(f"\nConfusion Matrix:")
    print(f"TN: {cm[0][0]}, FP: {cm[0][1]}")
    print(f"FN: {cm[1][0]}, TP: {cm[1][1]}")

    # Business metrics
    fpr = cm[0][1] / (cm[0][0] + cm[0][1])
    fnr = cm[1][0] / (cm[1][0] + cm[1][1])

    print(f"\n💼 Business Impact:")
    print(f"False Positive Rate: {fpr:.4f} (Target: <0.10)")
    print(f"False Negative Rate: {fnr:.4f} (Target: <0.15)")

    # Save model
    model.save('recyclo_fraud_detector_v2.h5')
    with open('feature_scaler_v2.pkl', 'wb') as f:
        pickle.dump(scaler, f)

    print("\n✅ Improved model saved!")

    return model, scaler, results, history

# Run improved training
if __name__ == "__main__":
    model, scaler, results, history = improved_training()

🚀 Starting IMPROVED Fraud Detection Training
🔄 Generating realistic training data with noise...
✅ Realistic dataset created: 10000 samples
📊 Legitimate: 7753, Fraud: 2247
📈 Class distribution: is_fraud
0    0.7753
1    0.2247
Name: proportion, dtype: float64

📊 Feature Analysis:
         similarity     time_diff  location_diff    user_score  \
count  10000.000000  10000.000000   10000.000000  10000.000000   
mean       0.730198     19.161910       5.586155      0.575040   
std        0.184078     47.059710      14.298398      0.229180   
min        0.101898      0.058523       0.001723      0.000207   
25%        0.649421      1.924437       0.392845      0.457821   
50%        0.763760      3.961086       1.159025      0.617367   
75%        0.861202     11.532417       3.444949      0.743135   
max        1.000000    335.847625      99.743230      0.988831   

       waste_type_match  quantity_ratio  lighting_score  angle_consistency  \
count      10000.000000    10000.000000    1000

Model: "RecycloFraudDetector_v2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ features (InputLayer)           │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 10)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │           704 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization             │ (None, 64)             │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_1           │ (None, 32)             │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 32)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 16)             │           528 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 16)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ fraud_prob (Dense)              │ (None, 1)              │            17 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,713 (14.50 KB)

 Trainable params: 3,521 (13.75 KB)

 Non-trainable params: 192 (768.00 B)

Epoch 1/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - accuracy: 0.6959 - loss: 5.8890 - precision: 0.3787 - recall: 0.5366 - val_accuracy: 0.9700 - val_loss: 4.9200 - val_precision: 1.0000 - val_recall: 0.8667 - learning_rate: 5.0000e-04
Epoch 2/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.8975 - loss: 4.7436 - precision: 0.7446 - recall: 0.8422 - val_accuracy: 0.9695 - val_loss: 4.0105 - val_precision: 0.9974 - val_recall: 0.8667 - learning_rate: 5.0000e-04
Epoch 3/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - accuracy: 0.9376 - loss: 3.8770 - precision: 0.8636 - recall: 0.8597 - val_accuracy: 0.9700 - val_loss: 3.1961 - val_precision: 0.9974 - val_recall: 0.8689 - learning_rate: 5.0000e-04
Epoch 4/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - accuracy: 0.9557 - loss: 3.0844 - precision: 0.9141 - recall: 0.8797 - val_accuracy: 0.9695 - val_loss: 2.4854 - val_precision: 0.9949 - val_recall: 0.8689 - learning_rate: 5.0000e-04
Epoch 5/200
94/94 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step 


🎯 REALISTIC MODEL EVALUATION
✅ Accuracy:  0.9740 (Target: 0.85-0.95)
🎯 Precision: 0.9401 (Target: 0.80-0.90)
📊 Recall:    0.9443 (Target: 0.75-0.90)
📈 F1-Score:  0.9422 (Target: 0.80-0.90)
📈 AUC Score: 0.9980 (Target: 0.85-0.95)

Confusion Matrix:
TN: 1524, FP: 27
FN: 25, TP: 424

💼 Business Impact:
False Positive Rate: 0.0174 (Target: <0.10)
False Negative Rate: 0.0557 (Target: <0.15)

✅ Improved model saved!
